# Scraping API Salesforce of IH

In [7]:
from simple_salesforce import Salesforce
import pandas as pd

sf = Salesforce(
username='anna.vilardell@ironhack.com', 
password='******', 
security_token='3qFYRVabawOuqETDj5ojHCrV')


# Opportunity table
desc = sf.Opportunity.describe()  
field_names = [field['name'] for field in desc['fields']]
soql_opp = "SELECT {} FROM Opportunity".format(','.join(field_names))
results_opp = sf.query_all(soql_opp)
sf_opp = pd.DataFrame.from_dict(results_opp["records"])

In [70]:
sf_opp.columns

Index(['attributes', 'Id', 'IsDeleted', 'AccountId', 'RecordTypeId', 'Name',
       'Description', 'StageName', 'Amount', 'Probability',
       ...
       'Owner_Name__c', 'Days_until_closure__c', 'Partnership_Discount__c',
       'Partnership_Discount_Amount__c', 'Qualified__c',
       'Thank_you_email_sent__c', 'Invoice_Number__c', 'Flywire_link__c',
       'Last_Payment_Date__c', 'Invoice_Date__c'],
      dtype='object', length=223)

In [81]:
list_selected = ['Id','RecordTypeId','Campus__c','Campus_Timezone__c','StageName', 'CloseDate','LeadSource', 'IsClosed', 'IsWon',
'CreatedDate','LastActivityDate', 'FiscalQuarter', 'FiscalYear','Bootcamp_Year__c','Bootcamp_Course__c','CI_StartDate__c',
'Start_Date_Month__c','Bootcamp_Start_Date__c',
'CI_bootcamp__c', 'CI_bootcamptype__c','Drop_Reason__c','Lost_Deal_Reason__c','Stage_Before_Lost__c',
'Discount_Type__c', 'Discount_Name__c','Discount_Amount__c',
 'utm_source__c','how_did_you_hear_about_us__c','Qty_Left__c',
'Bootcamp_Format__c', 'First_Touchpoint_type__c', 'First_touchpoint_Date__c',
'All_Traffic_Sources__c','gaconnector_First_Click_Source__c','gaconnector_First_Click_Medium__c',
'gaconnector_Last_Click_Source__c','Last_Click_Source_Medium__c','CurrencyIsoCode','Amount','Net_Amount__c',
'Balance_del__c', 'Balance_Financing__c','Payment_Plan__c','Financing_Option_Amount_Total__c', ]
new_opp = sf_opp[list_selected]
new_opp.columns = [col.lower().strip().replace(" ", "_") for col in new_opp.columns]

In [82]:
new_opp.head()

,id,recordtypeid,campus__c,campus_timezone__c,stagename,closedate,leadsource,isclosed,iswon,createddate,...,gaconnector_first_click_medium__c,gaconnector_last_click_source__c,last_click_source_medium__c,currencyisocode,amount,net_amount__c,balance_del__c,balance_financing__c,payment_plan__c,financing_option_amount_total__c
0,0061t000006zN63AAE,None,BCN,BCN,Paid Deposit,2019-03-13,None,True,True,2019-04-09T16:40:15.000+0000,...,None,None,/,EUR,7000.0,7000.0,7000.0,7000.0,None,0.0
1,0061t000006yv2OAAQ,None,BCN,BCN,Closed lost,2017-01-19,None,True,False,2016-07-18T18:19:19.000+0000,...,None,None,/,EUR,6500.0,5500.0,5500.0,5500.0,None,0.0
2,0061t000006yuqxAAA,None,MAD,MAD,Closed lost,2016-09-16,None,True,False,2016-05-20T23:13:16.000+0000,...,None,None,/,EUR,6500.0,6500.0,6500.0,6500.0,None,0.0
3,0061t000006yuqyAAA,None,MAD,MAD,Closed lost,2016-09-16,None,True,False,2016-05-21T01:48:01.000+0000,...,None,None,/,EUR,6500.0,6500.0,6500.0,6500.0,None,0.0
4,0061t000006yuqzAAA,None,MAD,MAD,Closed lost,2016-09-16,None,True,False,2016-05-21T20:54:26.000+0000,...,None,None,/,EUR,6500.0,6500.0,6500.0,6500.0,None,0.0


In [73]:
new_opp.shape

(90873, 44)

In [74]:
new_opp.recordtypeid.value_counts()

0121t000000QtE3AAK    40136
0125J000000D5zyQAC    14969
0121t000000QvN2AAK    13003
0121t000000QucfAAC     6961
0125J000000D5ztQAC     5105
0121t000000QtDyAAK     2938
0125J000000TqYrQAK     2000
0125J000000LSJ6QAO     1984
0125J000000TqYNQA0     1887
0125J000000LRLFQA4      614
0125J000000TqPQQA0      201
0125J000000TqYIQA0      189
0121t000000QvoIAAS       10
0121t000000QuvsAAC        3
0125J000000LSJGQA4        3
0121t000000QuvtAAC        2
Name: recordtypeid, dtype: int64

**We'll be analysing only organic application**, so RecordTypeId=0121t000000QtE3AAK, the rest correspond to diferent scholarships applications. 

In [85]:
new_opp=new_opp[new_opp["recordtypeid"]=="0121t000000QtE3AAK"]

In [86]:
new_opp.shape

(40136, 44)

In [87]:
new_opp.isnull().sum()

id                                       0
recordtypeid                             0
campus__c                              286
campus_timezone__c                     286
stagename                                0
closedate                                0
leadsource                           24306
isclosed                                 0
iswon                                    0
createddate                              0
lastactivitydate                     19028
fiscalquarter                            0
fiscalyear                               0
bootcamp_year__c                      3551
bootcamp_course__c                    3588
ci_startdate__c                      22625
start_date_month__c                   8476
bootcamp_start_date__c                3551
ci_bootcamp__c                       22408
ci_bootcamptype__c                   22417
drop_reason__c                       39763
lost_deal_reason__c                   8016
stage_before_lost__c                  9094
discount_ty

In [80]:
new_opp.head()

,id,recordtypeid,stagename,amount,closedate,leadsource,isclosed,iswon,createddate,lastactivitydate,...,all_traffic_sources__c,gaconnector_first_click_source__c,gaconnector_first_click_medium__c,gaconnector_last_click_source__c,last_click_source_medium__c,campus_timezone__c,balance_del__c,payment_plan__c,financing_option_amount_total__c,balance_financing__c
798,0061t00000JDZvjAAH,0121t000000QtE3AAK,Closed lost,70000.0,2019-10-05,ISA,True,False,2019-10-03T22:09:04.000+0000,None,...,None,None,None,None,/,MEX,70000.0,None,0.0,70000.0
799,0061t00000JDIzpAAH,0121t000000QtE3AAK,Closed lost,7500.0,2019-10-07,None,True,False,2019-10-02T15:58:41.000+0000,None,...,None,None,None,None,/,BCN,7500.0,None,0.0,7500.0
800,0061t00000JEXuJAAX,0121t000000QtE3AAK,Closed lost,7500.0,2019-10-07,pardot,True,False,2019-10-07T14:01:32.000+0000,None,...,None,None,None,None,/,AMS,7500.0,None,0.0,7500.0
802,0061t00000JDZyOAAX,0121t000000QtE3AAK,Closed lost,70000.0,2019-10-08,ISA,True,False,2019-10-04T00:16:11.000+0000,None,...,None,None,None,None,/,MEX,70000.0,None,0.0,70000.0
803,0061t00000JE4JiAAL,0121t000000QtE3AAK,Closed lost,NaN,2019-10-08,None,True,False,2019-10-06T00:31:24.000+0000,None,...,None,None,None,None,/,AMS,0.0,None,0.0,0.0


### We finally won't be using this dataset from our scraping as data was more compeleted when downloded directly from SF system